<img align="right" src="tf-small.png"/>

![mql](emdros.png)

# TF from MQL

This notebook can read an
[MQL](https://emdros.org/mql.html)
dump of a version of the [BHSA](https://github.com/ETCBC/bhsa) Hebrew Text Database
and transform it in a Text-Fabric
[Text-Fabric](https://github.com/ETCBC/text-fabric)
resource.

## Discussion

The principled way of going about such a conversion is to import the MQL source into
an [Emdros](https://emdros.org) database, and use it to retrieve objects and features from there.

Because the syntax of an MQL file leaves some freedom, it is error prone to do a text-to-text conversion from
MQL to something else.

Yet this is what we do, the error-prone thing. We then avoid installing and configuring and managing Emdros, MySQL/sqLite3.
Aside the upfront work to get this going, the going after that would also be much slower.

So here you are, a smallish script to do an awful lot of work, mostly correct, if careful used.

# Caveat

This notebook makes use of a new feature of text-fabric, first present in 2.3.12.
Make sure to upgrade first.

```sudo -H pip3 install --upgrade text-fabric
```

In [1]:
import os,sys,re,collections
from shutil import rmtree
from tf.fabric import Fabric
import utils

# Pipeline
See [operation](https://github.com/ETCBC/pipeline/blob/master/README.md#operation) 
for how to run this script in the pipeline.

In [2]:
if 'SCRIPT' not in locals():
    SCRIPT = False
    FORCE = True
    CORE_NAME = 'bhsa'
    VERSION = 'c'

def stop(good=False):
    if SCRIPT: sys.exit(0 if good else 1)

# Setting up the context: source file and target directories

The conversion is executed in an environment of directories, so that sources, temp files and
results are in convenient places and do not have to be shifted around.

In [3]:
repoBase = os.path.expanduser('~/github/etcbc')
thisRepo = '{}/{}'.format(repoBase, CORE_NAME)

thisSource = '{}/source/{}'.format(thisRepo, VERSION)
mqlzFile = '{}/{}.mql.bz2'.format(thisSource, CORE_NAME)

thisTemp = '{}/_temp/{}'.format(thisRepo, VERSION)
thisTempSource = '{}/source'.format(thisTemp)
mqlFile = '{}/{}.mql'.format(thisTempSource, CORE_NAME)
thisTempTf = '{}/tf'.format(thisTemp)

thisTf = '{}/tf/{}'.format(thisRepo, VERSION)

# Test

Check whether this conversion is needed in the first place.
Only when run as a script.

In [4]:
if SCRIPT:
    testFile = '{}/.tf/otype.tfx'.format(thisTf)
    (good, work) = utils.mustRun(mqlzFile, '{}/.tf/otype.tfx'.format(thisTf), force=FORCE)
    if not good: stop(good=False)
    if not work: stop(good=True)

# TF Settings

We add some custom information here.

* the MQL object type that corresponds to the TF slot type, typically `word`;
* a piece of metadata that will go into every feature; the time will be added automatically
* suitable text formats for the `otext` feature of TF.

The oText feature is very sensitive to what is available in the source MQL.
It needs to be configured here.
We save the configs we need per source and version.
And we define a stripped down default version to start with.

In [5]:
slotType = 'word'

featureMetaData = dict(
    dataset='BHSA',
    datasetName='Biblia Hebraica Stuttgartensia Amstelodamensis',
    author='Eep Talstra Centre for Bible and Computer',
    encoders='Constantijn Sikkel (QDF), Ulrik Petersen (MQL) and Dirk Roorda (TF)',
    website='https://shebanq.ancient-data.org',
    email='shebanq@ancient-data.org',
)

oText = {
    '': {
        '': '''
@sectionFeatures=book,chapter,verse
@sectionTypes=book,chapter,verse
@fmt:text-orig-full={g_word_utf8}{g_suffix_utf8}
''',
    },
    '4': '''
@fmt:lex-orig-full={g_lex_utf8} 
@fmt:lex-orig-plain={lex_utf8} 
@fmt:lex-trans-full={g_lex} 
@fmt:lex-trans-plain={lex} 
@fmt:text-orig-full={g_qere_utf8/g_word_utf8}{qtrailer_utf8/trailer_utf8}
@fmt:text-orig-full-ketiv={g_word_utf8}{trailer_utf8}
@fmt:text-orig-plain={g_cons_utf8}{trailer_utf8}
@fmt:text-trans-full={g_word} 
@fmt:text-trans-full-ketiv={g_word} 
@fmt:text-trans-plain={g_cons} 
@sectionFeatures=book,chapter,verse
@sectionTypes=book,chapter,verse
''',
    '4b': '''
@fmt:lex-orig-full={g_lex_utf8} 
@fmt:lex-orig-plain={lex_utf8} 
@fmt:lex-trans-full={g_lex} 
@fmt:lex-trans-plain={lex} 
@fmt:text-orig-full={g_qere_utf8/g_word_utf8}{qtrailer_utf8/trailer_utf8}
@fmt:text-orig-full-ketiv={g_word_utf8}{trailer_utf8}
@fmt:text-orig-plain={g_cons_utf8}{trailer_utf8}
@fmt:text-trans-full={g_word} 
@fmt:text-trans-full-ketiv={g_word} 
@fmt:text-trans-plain={g_cons} 
@sectionFeatures=book,chapter,verse
@sectionTypes=book,chapter,verse
''',
    'c': '''
@fmt:lex-orig-full={g_lex_utf8} 
@fmt:lex-orig-plain={lex_utf8} 
@fmt:lex-trans-full={g_lex} 
@fmt:lex-trans-plain={lex} 
@fmt:text-orig-full={g_word_utf8}{trailer_utf8}
@fmt:text-orig-plain={g_cons_utf8}{trailer_utf8}
@fmt:text-trans-full={g_word}{trailer}
@fmt:text-trans-plain={g_cons}{trailer}
@sectionFeatures=book,chapter,verse
@sectionTypes=book,chapter,verse
''',
    '2016': '''
@fmt:lex-orig-full={g_lex_utf8} 
@fmt:lex-orig-plain={lex_utf8} 
@fmt:lex-trans-full={g_lex} 
@fmt:lex-trans-plain={lex} 
@fmt:text-orig-full={g_word_utf8}{trailer_utf8}
@fmt:text-orig-plain={g_cons_utf8}{trailer_utf8}
@fmt:text-trans-full={g_word}{trailer}
@fmt:text-trans-plain={g_cons}{trailer}
@sectionFeatures=book,chapter,verse
@sectionTypes=book,chapter,verse
''',
    '2017': '''
@fmt:lex-orig-full={g_lex_utf8} 
@fmt:lex-orig-plain={lex_utf8} 
@fmt:lex-trans-full={g_lex} 
@fmt:lex-trans-plain={lex} 
@fmt:text-orig-full={g_word_utf8}{trailer_utf8}
@fmt:text-orig-plain={g_cons_utf8}{trailer_utf8}
@fmt:text-trans-full={g_word}{trailer}
@fmt:text-trans-plain={g_cons}{trailer}
@sectionFeatures=book,chapter,verse
@sectionTypes=book,chapter,verse
''',
}

The next function selects the proper otext material, falling back on a default if nothing 
appropriate has been specified in `oText`.

In [6]:
thisOtext = oText.get(VERSION, oText[''])

if thisOtext is oText['']:
    utils.caption(0, 'WARNING: no otext feature info provided, using a meager default value')
    otextInfo = {}
else:
    utils.caption(0, 'INFO: otext feature information found')
    otextInfo = dict(line[1:].split('=', 1) for line in thisOtext.strip('\n').split('\n'))
    for x in sorted(otextInfo.items()):
        utils.caption(0, '\t{:<20} = "{}"'.format(*x))

|       0.00s INFO: otext feature information found
|       0.00s 	fmt:lex-orig-full    = "{g_lex_utf8} "
|       0.00s 	fmt:lex-orig-plain   = "{lex_utf8} "
|       0.00s 	fmt:lex-trans-full   = "{g_lex} "
|       0.00s 	fmt:lex-trans-plain  = "{lex} "
|       0.00s 	fmt:text-orig-full   = "{g_word_utf8}{trailer_utf8}"
|       0.00s 	fmt:text-orig-plain  = "{g_cons_utf8}{trailer_utf8}"
|       0.00s 	fmt:text-trans-full  = "{g_word}{trailer}"
|       0.00s 	fmt:text-trans-plain = "{g_cons}{trailer}"
|       0.00s 	sectionFeatures      = "book,chapter,verse"
|       0.00s 	sectionTypes         = "book,chapter,verse"


# Overview

The program has several stages:
   
1. **prepare** the source (utils.bunzip if needed)
1. **parse MQL** and collect information in datastructures
1. **transform to TF** write the datastructures as TF features
1. **differences** (informational)
1. **deliver** the tf data at its destination directory
1. **compile** all tf features to binary format

# Prepare

Check the source, utils.bunzip it if needed, empty the result directory.

In [7]:
if not os.path.exists(thisTempSource):
    os.makedirs(thisTempSource)

utils.caption(0, 'bunzipping {} ...'.format(mqlzFile))
utils.bunzip(mqlzFile, mqlFile)
utils.caption(0, 'Done')

if os.path.exists(thisTempTf): rmtree(thisTempTf)
os.makedirs(thisTempTf)

|       0.02s bunzipping /Users/dirk/github/etcbc/bhsa/source/c/bhsa.mql.bz2 ...
|       0.02s 	NOTE: Using existing unzipped file which is newer than bzipped one
|       0.02s Done


Convert a monads specification (a comma separated sequence of numbers and number ranges)
into a set of integers.

# MQL to Text-Fabric
Transform the collected information in feature-like datastructures, and write it all
out to `.tf` files.

In [8]:
TF = Fabric(locations=thisTempTf, silent=True)
TF.importMQL(mqlFile, slotType=slotType, otext=otextInfo, meta=featureMetaData)
#importMQL(mqlFile, thisTempTf, slotType=slotType, otext=otextInfo, meta=featureMetadata)

  0.00s Parsing mql source ...
  0.00s 		enum boolean_t
  0.00s 		enum phrase_determination_t
  0.00s 		enum language_t
  0.01s 		enum book_name_t
  0.01s 		enum lexical_set_t
  0.01s 		enum verbal_stem_t
  0.01s 		enum verbal_tense_t
  0.01s 		enum person_t
  0.01s 		enum number_t
  0.02s 		enum gender_t
  0.02s 		enum state_t
  0.02s 		enum part_of_speech_t
  0.02s 		enum phrase_type_t
  0.02s 		enum phrase_atom_relation_t
  0.03s 		enum phrase_relation_t
  0.03s 		enum phrase_atom_unit_distance_to_mother_t
  0.03s 		enum subphrase_relation_t
  0.03s 		enum subphrase_mother_object_type_t
  0.03s 		enum phrase_function_t
  0.03s 		enum clause_atom_type_t
  0.04s 		enum clause_type_t
  0.04s 		enum clause_kind_t
  0.04s 		enum clause_constituent_relation_t
  0.04s 		enum clause_constituent_mother_object_type_t
  0.04s 		enum clause_constituent_unit_distance_to_mother_t
  0.04s 		otype word
  0.04s 			feature number (int) =def= 0 : node
  0.05s 			feature g_voc_lex (str) =def=  : node
 

 3m 04s 39 objects of type book
 3m 04s 88000 objects of type clause
 3m 04s 45180 objects of type half_verse
 3m 04s 23213 objects of type verse
 3m 04s 267515 objects of type phrase_atom
 3m 04s Making TF data ...
 3m 04s Monad - idd mapping ...
 3m 05s maxSlot=426581
 3m 05s Node mapping and otype ...
 3m 05s oslots ...
 3m 06s metadata ...
 3m 06s features ...
 3m 06s 	features from words
 3m 10s 	   100000 words
 3m 14s 	   200000 words
 3m 18s 	   300000 words
 3m 22s 	   400000 words
 3m 23s 	   426581 words
 3m 23s 	features from books
 3m 23s 	       39 books
 3m 23s 	features from chapters
 3m 23s 	      929 chapters
 3m 23s 	features from clauses
 3m 24s 	    88000 clauses
 3m 24s 	features from clause_atoms
 3m 26s 	    90562 clause_atoms
 3m 26s 	features from half_verses
 3m 26s 	    45180 half_verses
 3m 26s 	features from phrases
 3m 27s 	   100000 phrases
 3m 28s 	   200000 phrases
 3m 28s 	   253174 phrases
 3m 28s 	features from phrase_atoms
 3m 29s 	   100000 phrase

# Diffs

Check differences with previous versions.

The new dataset has been created in a temporary directory,
and has not yet been copied to its destination.

Here is your opportunity to compare the newly created features with the older features.
You expect some differences in some features.

We check the differences between the previous version of the features and what has been generated.
We list features that will be added and deleted and changed.
For each changed feature we show the first line where the new feature differs from the old one.
We ignore changes in the metadata, because the timestamp in the metadata will always change.

In [9]:
utils.checkDiffs(thisTempTf, thisTf)

..............................................................................................
.      5m 00s Check differences with previous version                                        .
..............................................................................................
|      5m 00s 	no features to add
|      5m 00s 	26 features to delete
|      5m 00s 		book@am
|      5m 00s 		book@ar
|      5m 00s 		book@bn
|      5m 00s 		book@da
|      5m 00s 		book@de
|      5m 00s 		book@el
|      5m 00s 		book@en
|      5m 00s 		book@es
|      5m 00s 		book@fa
|      5m 00s 		book@fr
|      5m 00s 		book@he
|      5m 00s 		book@hi
|      5m 00s 		book@id
|      5m 00s 		book@ja
|      5m 00s 		book@ko
|      5m 00s 		book@la
|      5m 00s 		book@nl
|      5m 00s 		book@pa
|      5m 00s 		book@pt
|      5m 00s 		book@ru
|      5m 00s 		book@sw
|      5m 00s 		book@syc
|      5m 00s 		book@tr
|      5m 00s 		book@ur
|      5m 00s 		book@yo
|      5m 00s 		book@zh
|      5m 00s 	69 f

# Deliver 

Copy the new TF dataset from the temporary location where it has been created to its final destination.

In [10]:
utils.deliverDataset(thisTempTf, thisTf)

..............................................................................................
.      8m 01s Deliver data set to /Users/dirk/github/etcbc/bhsa/tf/c                         .
..............................................................................................


# Compile TF

Just to see whether everything loads and the precomputing of extra information works out.
Moreover, if you want to work with these features, then the precomputing has already been done, and everything is quicker in subsequent runs.

We issue load statement to trigger the precomputing of extra data.
Note that all features specified text formats in the `otext` config feature,
will be loaded, as well as the features for sections.

At that point we have access to the full list of features.
We grab them and are going to load them all! 

In [ ]:
utils.caption(4, 'Load and compile standard TF features')
TF = Fabric(locations=thisTf, modules=[''])
api = TF.load('')

utils.caption(4, 'Load and compile all other TF features')
allFeatures = TF.explore(silent=False, show=True)
loadableFeatures = allFeatures['nodes'] + allFeatures['edges']
api = TF.load(loadableFeatures)
T = api.T

# Examples

In [11]:
utils.caption(4, 'Basic test')
utils.caption(4, 'First verse in all formats')
for fmt in T.formats:
    utils.caption(0, '{}'.format(fmt), continuation=True)
    utils.caption(0, '\t{}'.format(T.text(range(1,12), fmt=fmt)), continuation=True)

..............................................................................................
.      8m 08s Load and compile standard TF features                                          .
..............................................................................................
This is Text-Fabric 2.3.15
Api reference : https://github.com/ETCBC/text-fabric/wiki/Api
Tutorial      : https://github.com/ETCBC/text-fabric/blob/master/docs/tutorial.ipynb
Data sources  : https://github.com/ETCBC/text-fabric-data

69 features found and 0 ignored
  0.00s loading features ...
   |     0.83s T otype                from /Users/dirk/github/etcbc/bhsa/tf/c
   |     9.80s T oslots               from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.09s T book                 from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.05s T chapter              from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.04s T verse                from /Users/dirk/github/etcbc/bhsa/tf/c
   |     1.41s T g_cons        